In [10]:
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from nilearn import plotting
import numpy as np
import scipy

In [ ]:
# Some participants has less resting state sessions than others:
# subj - end 
#1 - 30
#2 - 30
#3 - 29
#4 - 27
#5 - 30
#6 - 29
#7 - 30
#8 - 27
# also the roi masks are saved in different places on the hub:
# subj 1: /home/jovyan/shared/NSD/nsddata/ppdata/subj01/func1pt8mm/roi/rh.MTL.nii.gz"
# subj 2-8: /home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roirh.MTL.nii.gz"

In [11]:
# loop through subjects 
last_sess = [30,30,29,27,30,29,30,27]
for subj_id in range(1,9):
    # 1.8mm MTL mask
    rMTL_= nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roirh.MTL.nii.gz")
    rMTL = rMTL_.get_fdata()

    lMTL_= nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roilh.MTL.nii.gz")
    lMTL = lMTL_.get_fdata()

    # 1.8mm floc masks
    rbodies_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roirh.floc-bodies.nii.gz")
    rbodies = rbodies_.get_fdata()

    rfaces_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roirh.floc-faces.nii.gz")
    rfaces = rfaces_.get_fdata()

    rwords_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roirh.floc-words.nii.gz")
    rwords = rwords_.get_fdata()

    lbodies_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roilh.floc-bodies.nii.gz")
    lbodies = lbodies_.get_fdata()

    lfaces_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roilh.floc-faces.nii.gz")
    lfaces = lfaces_.get_fdata()

    lwords_ = nib.load(f"/home/jovyan/shared/NSD/nsddata/ppdata/subj0{subj_id}/func1pt8mm/roilh.floc-words.nii.gz")
    lwords = lwords_.get_fdata()
    
    # compute ROI connectivity matrix for the first resting state across all sessions
    ROI_fc_all = np.zeros(shape=(26,26,10))
    for session in range(21,last_sess[subj_id-1]+1):
        # load the whole time series
        rs_ts = nib.load(f"/home/jovyan/shared/NSD/nsddata_timeseries/ppdata/subj0{subj_id}/func1pt8mm/timeseries/timeseries_session{session}_run01.nii.gz")
        ts = rs_ts.slicer[:,:,:,:].get_fdata() # get all timepoint 
        # get cordinates of where MTL mask label 
        all_region = np.zeros(shape=(226,26))

        # get the right MTL subregions
        for mask_lab in range(1,11):
            mask_cord = np.where(rMTL==mask_lab)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,mask_lab-1]= region_mean
        # get the left MTL subregions
        for mask_lab in range(1,11):
            mask_cord = np.where(lMTL==mask_lab)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,mask_lab+9]= region_mean
        # get the floc extraction
        n = 20
        for floc in [rbodies,rfaces,rwords,lbodies,lfaces,lwords]:
            mask_cord = np.where(floc>0)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,n]= region_mean
            n+=1

        ROI_fc = np.corrcoef(all_region,rowvar=False)
        ROI_fc_all[:,:,session-21]=ROI_fc
    np.save(f"./subj0{subj_id}_ROI_fc_first.npy", ROI_fc_all)
    # compute ROI connectivity matrix for the first resting state across all sessions
    ROI_fc_all = np.zeros(shape=(26,26,10))
    for session in range(21,last_sess[subj_id-1]+1):
        # load the whole time series
        rs_ts = nib.load(f"/home/jovyan/shared/NSD/nsddata_timeseries/ppdata/subj01/func1pt8mm/timeseries/timeseries_session{session}_run14.nii.gz")
        ts = rs_ts.slicer[:,:,:,:].get_fdata() # get all timepoint 
        # get cordinates of where MTL mask label 
        all_region = np.zeros(shape=(226,26))

        # get the right MTL subregions
        for mask_lab in range(1,11):
            mask_cord = np.where(rMTL==mask_lab)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,mask_lab-1]= region_mean
        # get the left MTL subregions
        for mask_lab in range(1,11):
            mask_cord = np.where(lMTL==mask_lab)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,mask_lab+9]= region_mean
        # get the floc extraction
        n = 20
        for floc in [rbodies,rfaces,rwords,lbodies,lfaces,lwords]:
            mask_cord = np.where(floc>0)
            region_ts = np.asarray(ts[mask_cord[0],mask_cord[1],mask_cord[2],:])
            region_mean = np.mean(region_ts,axis=0)
            all_region[:,n]= region_mean
            n+=1

        ROI_fc = np.corrcoef(all_region,rowvar=False)
        ROI_fc_all[:,:,session-21]=ROI_fc
    np.save(f"./subj0{subj_id}_ROI_fc_last.npy", ROI_fc_all)